# Making predictions with BlazeFace

This notebook shows how to use the model for face detection.

In [1]:
import numpy as np
import torch
import cv2

In [2]:
print("PyTorch version:", torch.__version__)
print("CUDA version:", torch.version.cuda)
print("cuDNN version:", torch.backends.cudnn.version())

PyTorch version: 1.3.1
CUDA version: None
cuDNN version: None


In [3]:
gpu = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
gpu

device(type='cpu')

Helper code for making plots:

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def plot_detections(img, detections, with_keypoints=True):
    fig, ax = plt.subplots(1, figsize=(10, 10))
    ax.grid(False)
    ax.imshow(img)
    
    if isinstance(detections, torch.Tensor):
        detections = detections.cpu().numpy()

    if detections.ndim == 1:
        detections = np.expand_dims(detections, axis=0)

    print("Found %d faces" % detections.shape[0])
        
    for i in range(detections.shape[0]):
        ymin = detections[i, 0] * img.shape[0]
        xmin = detections[i, 1] * img.shape[1]
        ymax = detections[i, 2] * img.shape[0]
        xmax = detections[i, 3] * img.shape[1]

        rect = patches.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                 linewidth=1, edgecolor="r", facecolor="none", 
                                 alpha=detections[i, 16])
        ax.add_patch(rect)

        if with_keypoints:
            for k in range(6):
                kp_x = detections[i, 4 + k*2    ] * img.shape[1]
                kp_y = detections[i, 4 + k*2 + 1] * img.shape[0]
                circle = patches.Circle((kp_x, kp_y), radius=0.5, linewidth=1, 
                                        edgecolor="lightskyblue", facecolor="none", 
                                        alpha=detections[i, 16])
                ax.add_patch(circle)
        
    plt.show()

## Load the model

In [5]:
from blazeface import BlazeFace

net = BlazeFace().to(gpu)
net.load_weights("blazeface.pth")
net.load_anchors("anchors.npy")

# Optionally change the thresholds:
net.min_score_thresh = 0.75
net.min_suppression_threshold = 0.3

## Make a prediction

The input image should be 128x128. BlazeFace will not automatically resize the image, you have to do this yourself!

In [6]:
img = cv2.imread("face2.png")
dim = (128, 128)
resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
img = cv2.cvtColor(resized, cv2.COLOR_BGR2RGB)

error: OpenCV(4.2.0) /Users/travis/build/skvark/opencv-python/opencv/modules/imgproc/src/resize.cpp:4045: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [ ]:
detections = net.predict_on_image(img)
detections.shape

In [ ]:
detections

In [ ]:
detections[:3]

In [ ]:
plot_detections(img, detections)

## Make prediction on a batch of images

Load the images into a NumPy array of size `(batch, 128, 128, 3)`. Note: You could also use a PyTorch tensor here, in which case the shape must be `(batch, 3, 128, 128)`.

In [ ]:
filenames = [ "1face.png", "3faces.png", "4faces.png" ]

x = np.zeros((len(filenames), 128, 128, 3), dtype=np.uint8)

for i, filename in enumerate(filenames):
    img = cv2.imread(filename)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    x[i] = img

In [ ]:
detections = net.predict_on_batch(x)
[d.shape for d in detections]

In [ ]:
detections

In [ ]:
plot_detections(x[0], detections[0])

In [ ]:
plot_detections(x[1], detections[1])

In [ ]:
plot_detections(x[2], detections[2])